In [1]:
import pandas as pd
import numpy as np

from pandas.api.types import is_string_dtype
from collections import defaultdict
from time import time

pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [2]:
port_clientes = pd.read_csv("../datasets/portfolio_clientes.csv", sep=",")
port_comunicados = pd.read_csv("../datasets/portfolio_comunicados.csv", sep=",")
port_geral = pd.read_csv("../datasets/portfolio_geral.csv", sep=",", nrows=500000, skiprows=range(1, 8000000, 40))
port_tpv = pd.read_csv("../datasets/portfolio_tpv.csv", sep=",")
#port_geral_contrato_status = pd.read_csv("../datasets/portfolio_geral.csv", sep=",", usecols=['contrato_id', 'dt_ref_portfolio', 'status_contrato'])

data_quitacao_contrato = pd.read_csv("../datasets/data_quitacao_contrato.csv", sep=",").set_index('CONTRATO_ID').to_dict()['DT_QUITACAO']
data_quitacao_contrato = {key : pd.to_datetime(val,format="%Y-%m-%d") if not pd.isna(val) else None 
                                                    for key, val in data_quitacao_contrato.items()}


In [3]:
def tratar_dataset(dataset):

    for coluna in dataset.columns:
        if is_string_dtype(dataset[coluna]):
            dataset[coluna] = dataset[coluna].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
            dataset[coluna] = dataset[coluna].str.upper()
            
    dataset.columns = map(str.upper, dataset.columns)
    
    return dataset


def converter_para_datetime(dataset, lst_colunas, formato):
    for coluna in lst_colunas:
        dataset[coluna] = pd.to_datetime(dataset[coluna], format=formato)

In [4]:
lst_datasets = [port_clientes, port_comunicados, port_geral, port_tpv]#, port_geral_contrato_status]

for dataset in lst_datasets:
    dataset = tratar_dataset(dataset)
    dataset.dropna(inplace=True)
    dataset.drop_duplicates(inplace=True)


In [5]:
converter_para_datetime(port_tpv, ["DT_TRANSACAO"], "%Y%m%d")
converter_para_datetime(port_geral, ["DT_REF_PORTFOLIO", "SAFRA", "DT_CONTRATO", "DT_DESEMBOLSO", "DT_VENCIMENTO", "DT_WO"], "%Y-%m-%d")
converter_para_datetime(port_comunicados, ["DT_REF_PORTFOLIO", "DATA_ACAO"], "%Y-%m-%d")
#converter_para_datetime(port_geral_contrato_status, ["DT_REF_PORTFOLIO"], "%Y-%m-%d")

In [6]:
port_clientes.drop(port_clientes.index[port_clientes["ESTADO"] == "ND"], inplace = True)
port_clientes.drop(port_clientes.index[port_clientes["SUBSEGMENTO"] == "NONE"], inplace = True)

In [7]:
lst_nr_suspensos = set(port_geral.query("STATUS_CONTRATO == 'SUSPENDED'").NR_DOCUMENTO.tolist())
lst_nr_pagos = set(port_geral.query("STATUS_CONTRATO == 'SETTLED'").NR_DOCUMENTO.tolist())

In [8]:
intersection_set = set.intersection(set(lst_nr_pagos), set(lst_nr_suspensos))

In [9]:
lst_nr_pago_sem_suspenso = lst_nr_pagos - lst_nr_suspensos

# PESSOAS QUE QUITARAM A DIVIDA E NUNCA FORAM SUSPENSAS

In [10]:
lst_nr_suspenso_sem_pagar = lst_nr_suspensos - lst_nr_pagos

# PESSOAS QUE FORAM SUSPENSAS E NUNCA PAGARAM A DIVIDA

In [11]:
port_comunicados.head()

,CONTRATO_ID,DT_REF_PORTFOLIO,DATA_ACAO,TIPO_ACAO,ACAO,STATUS
0,0D823DDFF1839EB938DCCC68586AE365,2021-01-16,2021-01-18,HSM,CAMPANHAOBSERVACAO,LIDO
1,C2F8A21671DBAEAA4157E03682C639E1,2021-05-22,2021-05-24,HSM,CAMPANHAOBSERVACAO,ENTREGUE
2,E19A222C233D46FF4984489C13AA1B98,2020-09-15,2020-09-15,HSM,CAMPANHAOBSERVACAO,NAO ENTREGUE
3,C24FC7630FE5B89928BA9478C2CDD9B2,2021-07-20,2021-07-20,HSM,CAMPANHAOBSERVACAO,NAO ENTREGUE
4,0ABE9C2F21E941F2BD3B086277E33EBC,2020-12-26,2020-12-28,HSM,CAMPANHAOBSERVACAO,ENTREGUE


In [12]:
contratos_pagos = port_geral.loc[port_geral['NR_DOCUMENTO'].isin(lst_nr_pago_sem_suspenso)]
contratos_pagos['PERFIL_PAGADOR'] = 'BOM'

/tmp/ipykernel_6029/2733006254.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  contratos_pagos['PERFIL_PAGADOR'] = 'BOM'


In [13]:
contratos_pagos_suspensos = port_geral.loc[port_geral['NR_DOCUMENTO'].isin(intersection_set)]
contratos_pagos_suspensos['PERFIL_PAGADOR'] = 'NEUTRO'

/tmp/ipykernel_6029/65916991.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  contratos_pagos_suspensos['PERFIL_PAGADOR'] = 'NEUTRO'


In [14]:
contratos_suspensos = port_geral.loc[port_geral['NR_DOCUMENTO'].isin(lst_nr_suspenso_sem_pagar)]
contratos_suspensos['PERFIL_PAGADOR'] = 'MAU'

/tmp/ipykernel_6029/1349424356.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  contratos_suspensos['PERFIL_PAGADOR'] = 'MAU'


############################################################################


In [15]:
clientes_com_contratos_pago = contratos_pagos.merge(port_clientes, how='inner', on='NR_DOCUMENTO')

In [16]:
contratos_pagos_suspensos_merge = contratos_pagos_suspensos.merge(port_clientes, how='inner', on='NR_DOCUMENTO')

In [17]:
clientes_com_contratos_suspensos = contratos_suspensos.merge(port_clientes, how='inner', on='NR_DOCUMENTO')

In [18]:
todos_clientes = pd.concat([clientes_com_contratos_pago, contratos_pagos_suspensos_merge, clientes_com_contratos_suspensos])

############################################################################

In [19]:
def agrupa_e_segmenta_porcentagem(dataframe, coluna_groupby, coluna_tipo):
    tipos = list(filter(lambda elem: not pd.isna(elem), dataframe[coluna_tipo].unique()))
    
    tipos_dict = {}
    for tipo in tipos:
        query = dataframe.query(f'{coluna_tipo} == "{tipo}"')
        contagem = query.groupby(coluna_groupby).size()
        porcentagem = round(contagem / contagem.sum() * 100, 2)

        tipo_dataframe = pd.DataFrame()
        tipo_dataframe['contagem'] = contagem
        tipo_dataframe['porcentagem'] = porcentagem

        tipos_dict[tipo] = tipo_dataframe.sort_values(by=['contagem'], ascending=False)

    return tipos_dict

In [20]:
def gera_df_porcentagem_simples(dataframe, tipo):
    contagem = dataframe.drop_duplicates('NR_DOCUMENTO').groupby(tipo).size()
    porcentagem = round(contagem / contagem.sum() * 100,2)

    dataframe_resultado = pd.DataFrame()
    dataframe_resultado['contagem'] = contagem
    dataframe_resultado['porcentagem'] = porcentagem

    return dataframe_resultado.sort_values('contagem', ascending=False)

def gera_df_porcentagem_composto(dataframe, tipo, subtipo):
    resultado = defaultdict(list)
    agrupamento = dataframe.drop_duplicates('NR_DOCUMENTO').groupby([tipo, subtipo])
    contagem_total_por_tipo = agrupamento.size()

    for (tipo_aux, subtipo_aux), valor in agrupamento:
        contagem = len(valor)
        contagem_total = contagem_total_por_tipo.loc[tipo_aux].sum()
        resultado['tipo'].append(tipo_aux)
        resultado['subtipo'].append(subtipo_aux)
        resultado['contagem'].append(contagem)
        resultado['porcentagem'].append(round(contagem / contagem_total * 100,2))

    return pd.DataFrame(resultado).set_index(['tipo', 'subtipo']).sort_values(by=['tipo','contagem'], ascending=False)

def gera_df_porcentagem(dataframe, tipo, subtipo = None):
    resultado = gera_df_porcentagem_composto(dataframe, tipo, subtipo) if subtipo is not None else gera_df_porcentagem_simples(dataframe, tipo)

    return resultado

def gera_df_porcentagem_por_perfil_pagador(todos_perfis_pagadores_dataframe, tipo):
    return gera_df_porcentagem_composto(dataframe=todos_perfis_pagadores_dataframe, tipo=tipo, subtipo='PERFIL_PAGADOR')

def extrair_subtipo_ordenado(dataframe, subtipo):
    return dataframe.query(f'subtipo == "{subtipo}"').sort_values(by='porcentagem', ascending=False)
    

## TODO
- [x] Listar os agrupamentos por perfil de pagador (estado, tipo de empresa, segmento, status do contrato)
- [x] Listar demais tipos de agrupamentos (Estado x Segmento, Status do contrato x Tipo de empresa, Segmento x Status de contrato)
- [x] Adicionar resultados para perfil de pagador neutro
- [] Investigar a criação da curva de acionamento (plotar uma para cada perfil de pagador)
- [] Há um diferença entre a quantidade de lidos de um bom e mau pagador

## Bom pagador

### Tipo empresa

In [21]:
bom_pagador_tipo_empresa = gera_df_porcentagem(clientes_com_contratos_pago, 'TIPO_EMPRESA')
bom_pagador_tipo_empresa

,contagem,porcentagem
TIPO_EMPRESA,,
PJ,3950,59.26
MEI,2136,32.04
PF,580,8.70


### Segmento

#### Quantidade de bons pagadores por segmento

In [22]:
bom_pagador_segmento = gera_df_porcentagem(clientes_com_contratos_pago, 'SEGMENTO')
bom_pagador_segmento

,contagem,porcentagem
SEGMENTO,,
ALIMENTACAO,2474,37.11
VAREJO,1180,17.70
BENS DURAVEIS,1057,15.86
SERVICOS,1022,15.33
SUPERMERCADO/FARMACIA,627,9.41
SERVICOS RECORRENTES,139,2.09
VIAGENS E ENTRETENIMENTO,91,1.37
POSTO,76,1.14


#### Distribuição de status de contrato por segmento

In [23]:
bom_pagador_segmento_status = gera_df_porcentagem(clientes_com_contratos_pago, 'SEGMENTO', 'STATUS_CONTRATO')
bom_pagador_segmento_status

contagem  porcentagem
tipo                     subtipo                                     
VIAGENS E ENTRETENIMENTO ACTIVE                       46        50.55
                         SETTLED                      45        49.45
VAREJO                   ACTIVE                      637        53.98
                         SETTLED                     543        46.02
SUPERMERCADO/FARMACIA    ACTIVE                      364        58.05
                         SETTLED                     262        41.79
                         ACCEPTED                      1         0.16
SERVICOS RECORRENTES     SETTLED                      71        51.08
                         ACTIVE                       68        48.92
SERVICOS                 ACTIVE                      564        55.19
                         SETTLED                     457        44.72
                         DISBURSEMENTREQUESTED         1         0.10
POSTO                    ACTIVE                       42        55.26
                         SETTLED                      34        44.74
BENS DURAVEIS            ACTIVE                      551        52.13
                         SETTLED                     505        47.78
                         DISBURSEMENTREQUESTED         1         0.09
ALIMENTACAO              ACTIVE                     1350        54.57
                         SETTLED                    1121        45.31
                         DISBURSEMENTREQUESTED         2         0.08
                         DISBURSEMENTCONFIRMED         1         0.04

Qual segmento possui a maior taxa de contratos quitados?

In [58]:
bom_pagador_segmento_status_quitado = extrair_subtipo_ordenado(bom_pagador_segmento_status, 'SETTLED')
bom_pagador_segmento_status_quitado

,,contagem,porcentagem
tipo,subtipo,,
SERVICOS RECORRENTES,SETTLED,71,51.08
VIAGENS E ENTRETENIMENTO,SETTLED,45,49.45
BENS DURAVEIS,SETTLED,505,47.78
VAREJO,SETTLED,543,46.02
ALIMENTACAO,SETTLED,1121,45.31
POSTO,SETTLED,34,44.74
SERVICOS,SETTLED,457,44.72
SUPERMERCADO/FARMACIA,SETTLED,262,41.79


### Status contrato

In [25]:
bom_pagador_status_contrato = gera_df_porcentagem(clientes_com_contratos_pago, 'STATUS_CONTRATO')
bom_pagador_status_contrato

,contagem,porcentagem
STATUS_CONTRATO,,
ACTIVE,3622,54.34
SETTLED,3038,45.57
DISBURSEMENTREQUESTED,4,0.06
ACCEPTED,1,0.02
DISBURSEMENTCONFIRMED,1,0.02


### Estado

#### Quantidade de bons pagadores por estado

In [26]:
bom_pagador_estado = gera_df_porcentagem(clientes_com_contratos_pago, 'ESTADO')
bom_pagador_estado

,contagem,porcentagem
ESTADO,,
SP,1838,27.57
RJ,800,12.00
PR,620,9.30
MG,419,6.29
SC,395,5.93
RS,360,5.40
BA,344,5.16
GO,275,4.13
PE,205,3.08


#### Distribuição dos status de contrato por estado

In [27]:
bom_pagador_estado_status = gera_df_porcentagem(clientes_com_contratos_pago, 'ESTADO', 'STATUS_CONTRATO')
bom_pagador_estado_status

contagem  porcentagem
tipo subtipo                                     
TO   ACTIVE                       15        60.00
     SETTLED                      10        40.00
SP   ACTIVE                      968        52.67
     SETTLED                     867        47.17
     DISBURSEMENTREQUESTED         2         0.11
     DISBURSEMENTCONFIRMED         1         0.05
SE   ACTIVE                       40        62.50
     SETTLED                      24        37.50
SC   ACTIVE                      210        53.16
     SETTLED                     185        46.84
RS   ACTIVE                      189        52.50
     SETTLED                     171        47.50
RR   ACTIVE                       10        55.56
     SETTLED                       8        44.44
RO   SETTLED                      16        51.61
     ACTIVE                       15        48.39
RN   ACTIVE                       62        63.27
     SETTLED                      36        36.73
RJ   ACTIVE                      428        53.50
     SETTLED                     370        46.25
     DISBURSEMENTREQUESTED         2         0.25
PR   ACTIVE                      354        57.10
     SETTLED                     266        42.90
PI   SETTLED                      17        53.12
     ACTIVE                       15        46.88
PE   ACTIVE                      111        54.15
     SETTLED                      94        45.85
PB   ACTIVE                       44        57.89
     SETTLED                      32        42.11
PA   ACTIVE                       64        52.03
     SETTLED                      59        47.97
MT   ACTIVE                       61        62.89
     SETTLED                      36        37.11
MS   ACTIVE                       45        56.96
     SETTLED                      34        43.04
MG   ACTIVE                      228        54.42
     SETTLED                     190        45.35
     ACCEPTED                      1         0.24
MA   SETTLED                      35        60.34
     ACTIVE                       23        39.66
GO   ACTIVE                      153        55.64
     SETTLED                     122        44.36
ES   ACTIVE                       88        61.11
     SETTLED                      56        38.89
DF   ACTIVE                      106        52.74
     SETTLED                      95        47.26
CE   ACTIVE                      108        56.84
     SETTLED                      82        43.16
BA   ACTIVE                      192        55.81
     SETTLED                     152        44.19
AP   ACTIVE                       18        56.25
     SETTLED                      14        43.75
AM   ACTIVE                       35        57.38
     SETTLED                      26        42.62
AL   ACTIVE                       36        51.43
     SETTLED                      34        48.57
AC   SETTLED                       7        63.64
     ACTIVE                        4        36.36

Qual estado possui a maior taxa de contratos quitados?

In [59]:
bom_pagador_estado_status_quitados = extrair_subtipo_ordenado(bom_pagador_estado_status, 'SETTLED')
bom_pagador_estado_status_quitados

,,contagem,porcentagem
tipo,subtipo,,
AC,SETTLED,7,63.64
MA,SETTLED,35,60.34
PI,SETTLED,17,53.12
RO,SETTLED,16,51.61
AL,SETTLED,34,48.57
PA,SETTLED,59,47.97
RS,SETTLED,171,47.50
DF,SETTLED,95,47.26
SP,SETTLED,867,47.17


#### Distribuição de segmento de atuação por estado

In [29]:
bom_pagador_estado_segmento = gera_df_porcentagem(clientes_com_contratos_pago, 'ESTADO', 'SEGMENTO')
bom_pagador_estado_segmento

contagem  porcentagem
tipo subtipo                                        
TO   ALIMENTACAO                      8        32.00
     BENS DURAVEIS                    5        20.00
     SERVICOS                         5        20.00
     SUPERMERCADO/FARMACIA            5        20.00
     POSTO                            1         4.00
     SERVICOS RECORRENTES             1         4.00
SP   ALIMENTACAO                    748        40.70
     VAREJO                         303        16.49
     SERVICOS                       291        15.83
     BENS DURAVEIS                  282        15.34
     SUPERMERCADO/FARMACIA          132         7.18
     SERVICOS RECORRENTES            34         1.85
     VIAGENS E ENTRETENIMENTO        25         1.36
     POSTO                           23         1.25
SE   ALIMENTACAO                     20        31.25
     SERVICOS                        15        23.44
     VAREJO                          10        15.62
     BENS DURAVEIS                    9        14.06
     SUPERMERCADO/FARMACIA            8        12.50
     SERVICOS RECORRENTES             2         3.12
SC   ALIMENTACAO                    147        37.22
     VAREJO                          75        18.99
     SERVICOS                        65        16.46
     BENS DURAVEIS                   52        13.16
     SUPERMERCADO/FARMACIA           36         9.11
     SERVICOS RECORRENTES            10         2.53
     VIAGENS E ENTRETENIMENTO         6         1.52
     POSTO                            4         1.01
RS   ALIMENTACAO                    118        32.78
     SERVICOS                        58        16.11
     VAREJO                          57        15.83
     BENS DURAVEIS                   56        15.56
     SUPERMERCADO/FARMACIA           53        14.72
     SERVICOS RECORRENTES            11         3.06
     POSTO                            5         1.39
     VIAGENS E ENTRETENIMENTO         2         0.56
RR   SERVICOS                         5        27.78
     ALIMENTACAO                      4        22.22
     BENS DURAVEIS                    3        16.67
     SUPERMERCADO/FARMACIA            3        16.67
     VAREJO                           3        16.67
RO   BENS DURAVEIS                    9        29.03
     ALIMENTACAO                      8        25.81
     VAREJO                           8        25.81
     SERVICOS                         4        12.90
     POSTO                            2         6.45
RN   ALIMENTACAO                     30        30.61
     VAREJO                          25        25.51
     SERVICOS                        15        15.31
     BENS DURAVEIS                   13        13.27
     SUPERMERCADO/FARMACIA            8         8.16
     POSTO                            3         3.06
     SERVICOS RECORRENTES             2         2.04
     VIAGENS E ENTRETENIMENTO         2         2.04
RJ   ALIMENTACAO                    303        37.88
     VAREJO                         155        19.38
     BENS DURAVEIS                  126        15.75
     SERVICOS                       116        14.50
     SUPERMERCADO/FARMACIA           68         8.50
     SERVICOS RECORRENTES            15         1.88
     VIAGENS E ENTRETENIMENTO        13         1.62
     POSTO                            4         0.50
PR   ALIMENTACAO                    251        40.48
     SERVICOS                        97        15.65
     BENS DURAVEIS                   96        15.48
     VAREJO                          92        14.84
     SUPERMERCADO/FARMACIA           57         9.19
     SERVICOS RECORRENTES            13         2.10
     VIAGENS E ENTRETENIMENTO        10         1.61
     POSTO                            4         0.65
PI   ALIMENTACAO                     12        37.50
     BENS DURAVEIS                    7        21.88
     SUPERMERCADO/FARMACIA            5        15.62
     SERVICOS                         4        12.50
   

## Mau pagador

### Tipo empresa

In [42]:
mau_pagador_tipo_empresa = gera_df_porcentagem(clientes_com_contratos_suspensos, 'TIPO_EMPRESA')
mau_pagador_tipo_empresa

,contagem,porcentagem
TIPO_EMPRESA,,
PJ,561,51.05
MEI,368,33.48
PF,170,15.47


### Segmento

#### Quantidade de maus pagadores por segmento

In [33]:
mau_pagador_segmento = gera_df_porcentagem(clientes_com_contratos_suspensos, 'SEGMENTO')
mau_pagador_segmento

,contagem,porcentagem
SEGMENTO,,
ALIMENTACAO,332,30.21
VAREJO,234,21.29
SERVICOS,200,18.20
BENS DURAVEIS,196,17.83
SUPERMERCADO/FARMACIA,75,6.82
SERVICOS RECORRENTES,34,3.09
VIAGENS E ENTRETENIMENTO,17,1.55
POSTO,11,1.00


#### Distribuição de status de contrato por segmento

In [34]:
mau_pagador_segmento_status = gera_df_porcentagem(clientes_com_contratos_suspensos, 'SEGMENTO', 'STATUS_CONTRATO')
mau_pagador_segmento_status

contagem  porcentagem
tipo                     subtipo                                     
VIAGENS E ENTRETENIMENTO ACTIVE                       15        88.24
                         SUSPENDED                     2        11.76
VAREJO                   ACTIVE                      183        78.21
                         SUSPENDED                    51        21.79
SUPERMERCADO/FARMACIA    ACTIVE                       57        76.00
                         SUSPENDED                    17        22.67
                         DISBURSEMENTCONFIRMED         1         1.33
SERVICOS RECORRENTES     ACTIVE                       29        85.29
                         SUSPENDED                     5        14.71
SERVICOS                 ACTIVE                      160        80.00
                         SUSPENDED                    40        20.00
POSTO                    ACTIVE                       11       100.00
BENS DURAVEIS            ACTIVE                      150        76.53
                         SUSPENDED                    46        23.47
ALIMENTACAO              ACTIVE                      267        80.42
                         SUSPENDED                    65        19.58

Qual segmento possui a maior taxa de contratos suspensos?

In [56]:
mau_pagador_segmento_status_suspenso = extrair_subtipo_ordenado(mau_pagador_segmento_status, 'SUSPENDED')
mau_pagador_segmento_status_suspenso

,,contagem,porcentagem
tipo,subtipo,,
BENS DURAVEIS,SUSPENDED,46,23.47
SUPERMERCADO/FARMACIA,SUSPENDED,17,22.67
VAREJO,SUSPENDED,51,21.79
SERVICOS,SUSPENDED,40,20.00
ALIMENTACAO,SUSPENDED,65,19.58
SERVICOS RECORRENTES,SUSPENDED,5,14.71
VIAGENS E ENTRETENIMENTO,SUSPENDED,2,11.76


### Estado

#### Quantidade de maus pagadores por estado

In [37]:
mau_pagador_estado = gera_df_porcentagem(clientes_com_contratos_suspensos, 'ESTADO')
mau_pagador_estado

,contagem,porcentagem
ESTADO,,
SP,280,25.48
RJ,156,14.19
PR,91,8.28
BA,63,5.73
MG,57,5.19
GO,53,4.82
SC,49,4.46
DF,48,4.37
RS,46,4.19


#### Distribuição dos status de contrato por estado

In [38]:
mau_pagador_estado_status = gera_df_porcentagem(clientes_com_contratos_suspensos, 'ESTADO', 'STATUS_CONTRATO')
mau_pagador_estado_status

contagem  porcentagem
tipo subtipo                                     
TO   ACTIVE                        6       100.00
SP   ACTIVE                      227        81.07
     SUSPENDED                    53        18.93
SE   ACTIVE                        4        80.00
     SUSPENDED                     1        20.00
SC   ACTIVE                       36        73.47
     SUSPENDED                    13        26.53
RS   ACTIVE                       34        73.91
     SUSPENDED                    12        26.09
RR   ACTIVE                        5       100.00
RO   ACTIVE                        3        60.00
     SUSPENDED                     2        40.00
RN   ACTIVE                       14        77.78
     SUSPENDED                     4        22.22
RJ   ACTIVE                      116        74.36
     SUSPENDED                    40        25.64
PR   ACTIVE                       76        83.52
     SUSPENDED                    15        16.48
PI   ACTIVE                        4        80.00
     SUSPENDED                     1        20.00
PE   ACTIVE                       24        66.67
     SUSPENDED                    12        33.33
PB   ACTIVE                       10        76.92
     SUSPENDED                     3        23.08
PA   ACTIVE                       17        89.47
     SUSPENDED                     2        10.53
MT   ACTIVE                        7        63.64
     SUSPENDED                     4        36.36
MS   ACTIVE                       11        78.57
     SUSPENDED                     3        21.43
MG   ACTIVE                       45        78.95
     SUSPENDED                    12        21.05
MA   ACTIVE                       13        92.86
     SUSPENDED                     1         7.14
GO   ACTIVE                       44        83.02
     SUSPENDED                     9        16.98
ES   ACTIVE                       21        87.50
     SUSPENDED                     3        12.50
DF   ACTIVE                       38        79.17
     SUSPENDED                    10        20.83
CE   ACTIVE                       35        79.55
     SUSPENDED                     9        20.45
BA   ACTIVE                       52        82.54
     SUSPENDED                    10        15.87
     DISBURSEMENTCONFIRMED         1         1.59
AP   ACTIVE                        4        57.14
     SUSPENDED                     3        42.86
AM   ACTIVE                       13        81.25
     SUSPENDED                     3        18.75
AL   ACTIVE                       13        92.86
     SUSPENDED                     1         7.14

Qual estado possui a maior taxa de contratos suspensos?

In [57]:
mau_pagador_estado_status_suspenso = extrair_subtipo_ordenado(mau_pagador_estado_status, 'SUSPENDED')
mau_pagador_estado_status_suspenso

,,contagem,porcentagem
tipo,subtipo,,
AP,SUSPENDED,3,42.86
RO,SUSPENDED,2,40.00
MT,SUSPENDED,4,36.36
PE,SUSPENDED,12,33.33
SC,SUSPENDED,13,26.53
RS,SUSPENDED,12,26.09
RJ,SUSPENDED,40,25.64
PB,SUSPENDED,3,23.08
RN,SUSPENDED,4,22.22


#### Distribuição de segmento de atuação por estado

In [40]:
mau_pagador_estado_segmento = gera_df_porcentagem(clientes_com_contratos_suspensos, 'ESTADO', 'SEGMENTO')
mau_pagador_estado_segmento

contagem  porcentagem
tipo subtipo                                        
TO   BENS DURAVEIS                    3        50.00
     ALIMENTACAO                      2        33.33
     VAREJO                           1        16.67
SP   ALIMENTACAO                     81        28.93
     SERVICOS                        56        20.00
     VAREJO                          54        19.29
     BENS DURAVEIS                   51        18.21
     SUPERMERCADO/FARMACIA           20         7.14
     SERVICOS RECORRENTES             9         3.21
     VIAGENS E ENTRETENIMENTO         6         2.14
     POSTO                            3         1.07
SE   BENS DURAVEIS                    2        40.00
     SUPERMERCADO/FARMACIA            2        40.00
     ALIMENTACAO                      1        20.00
SC   ALIMENTACAO                     17        34.69
     BENS DURAVEIS                   10        20.41
     VAREJO                           8        16.33
     SERVICOS                         7        14.29
     SUPERMERCADO/FARMACIA            4         8.16
     SERVICOS RECORRENTES             3         6.12
RS   ALIMENTACAO                     16        34.78
     SERVICOS                        15        32.61
     BENS DURAVEIS                    6        13.04
     VAREJO                           6        13.04
     SUPERMERCADO/FARMACIA            2         4.35
     SERVICOS RECORRENTES             1         2.17
RR   ALIMENTACAO                      2        40.00
     BENS DURAVEIS                    1        20.00
     SERVICOS                         1        20.00
     SUPERMERCADO/FARMACIA            1        20.00
RO   SERVICOS                         3        60.00
     ALIMENTACAO                      2        40.00
RN   VAREJO                           8        44.44
     BENS DURAVEIS                    5        27.78
     SERVICOS                         2        11.11
     SUPERMERCADO/FARMACIA            2        11.11
     SERVICOS RECORRENTES             1         5.56
RJ   ALIMENTACAO                     49        31.41
     VAREJO                          34        21.79
     SERVICOS                        28        17.95
     BENS DURAVEIS                   21        13.46
     SUPERMERCADO/FARMACIA           14         8.97
     SERVICOS RECORRENTES             7         4.49
     VIAGENS E ENTRETENIMENTO         3         1.92
PR   VAREJO                          23        25.27
     ALIMENTACAO                     22        24.18
     SERVICOS                        21        23.08
     BENS DURAVEIS                   15        16.48
     SERVICOS RECORRENTES             3         3.30
     SUPERMERCADO/FARMACIA            3         3.30
     VIAGENS E ENTRETENIMENTO         3         3.30
     POSTO                            1         1.10
PI   ALIMENTACAO                      2        40.00
     SERVICOS                         1        20.00
     VAREJO                           1        20.00
     VIAGENS E ENTRETENIMENTO         1        20.00
PE   BENS DURAVEIS                   10        27.78
     ALIMENTACAO                      9        25.00
     VAREJO                           9        25.00
     SERVICOS                         7        19.44
     SUPERMERCADO/FARMACIA            1         2.78
PB   BENS DURAVEIS                    4        30.77
     VAREJO                           3        23.08
     ALIMENTACAO                      2        15.38
     SERVICOS                         2        15.38
     SUPERMERCADO/FARMACIA            1         7.69
     VIAGENS E ENTRETENIMENTO         1         7.69
PA   ALIMENTACAO                      4        21.05
     BENS DURAVEIS                    4        21.05
     SERVICOS                         3        15.79
     VAREJO                           3        15.79
     POSTO                            2        10.53
     SERVICOS RECORRENTES             1         5.26
     SUPERMERCADO/FARMACIA            1         5.26
   

### Status contrato

In [43]:
mau_pagador_status_contrato = gera_df_porcentagem(clientes_com_contratos_suspensos, 'STATUS_CONTRATO')
mau_pagador_status_contrato

,contagem,porcentagem
STATUS_CONTRATO,,
ACTIVE,872,79.34
SUSPENDED,226,20.56
DISBURSEMENTCONFIRMED,1,0.09


## Pagador neutro

### Tipo empresa

In [45]:
neutro_pagador_tipo_empresa = gera_df_porcentagem(contratos_pagos_suspensos_merge, 'TIPO_EMPRESA')
neutro_pagador_tipo_empresa

,contagem,porcentagem
TIPO_EMPRESA,,
PJ,503,56.71
MEI,292,32.92
PF,92,10.37


### Segmento

#### Quantidade de pagadores neutros por segmento

In [46]:
neutro_pagador_segmento = gera_df_porcentagem(contratos_pagos_suspensos_merge, 'SEGMENTO')
neutro_pagador_segmento

,contagem,porcentagem
SEGMENTO,,
ALIMENTACAO,313,35.29
SERVICOS,160,18.04
VAREJO,158,17.81
BENS DURAVEIS,148,16.69
SUPERMERCADO/FARMACIA,59,6.65
SERVICOS RECORRENTES,25,2.82
VIAGENS E ENTRETENIMENTO,15,1.69
POSTO,9,1.01


#### Distribuição de status de contrato por segmento

In [47]:
neutro_pagador_segmento_status = gera_df_porcentagem(contratos_pagos_suspensos_merge, 'SEGMENTO', 'STATUS_CONTRATO')
neutro_pagador_segmento_status

contagem  porcentagem
tipo                     subtipo                         
VIAGENS E ENTRETENIMENTO SETTLED           7        46.67
                         ACTIVE            6        40.00
                         SUSPENDED         2        13.33
VAREJO                   ACTIVE           74        46.84
                         SETTLED          63        39.87
                         SUSPENDED        21        13.29
SUPERMERCADO/FARMACIA    ACTIVE           25        42.37
                         SETTLED          23        38.98
                         SUSPENDED        11        18.64
SERVICOS RECORRENTES     ACTIVE           11        44.00
                         SETTLED          11        44.00
                         SUSPENDED         3        12.00
SERVICOS                 ACTIVE           92        57.50
                         SETTLED          42        26.25
                         SUSPENDED        26        16.25
POSTO                    SETTLED           6        66.67
                         SUSPENDED         2        22.22
                         ACTIVE            1        11.11
BENS DURAVEIS            ACTIVE           71        47.97
                         SETTLED          54        36.49
                         SUSPENDED        23        15.54
ALIMENTACAO              ACTIVE          170        54.31
                         SETTLED         108        34.50
                         SUSPENDED        35        11.18

Qual segmento possui a maior taxa de contratos quitados?

In [49]:
neutro_pagador_segmento_status_quitado = extrair_subtipo_ordenado(neutro_pagador_segmento_status, 'SETTLED')
neutro_pagador_segmento_status_quitado

,,contagem,porcentagem
tipo,subtipo,,
POSTO,SETTLED,6,66.67
VIAGENS E ENTRETENIMENTO,SETTLED,7,46.67
SERVICOS RECORRENTES,SETTLED,11,44.00
VAREJO,SETTLED,63,39.87
SUPERMERCADO/FARMACIA,SETTLED,23,38.98
BENS DURAVEIS,SETTLED,54,36.49
ALIMENTACAO,SETTLED,108,34.50
SERVICOS,SETTLED,42,26.25


Qual segmento possui a maior taxa de contratos suspensos?

In [50]:
neutro_pagador_segmento_status_suspenso = extrair_subtipo_ordenado(neutro_pagador_segmento_status, 'SUSPENDED')
neutro_pagador_segmento_status_suspenso

,,contagem,porcentagem
tipo,subtipo,,
POSTO,SUSPENDED,2,22.22
SUPERMERCADO/FARMACIA,SUSPENDED,11,18.64
SERVICOS,SUSPENDED,26,16.25
BENS DURAVEIS,SUSPENDED,23,15.54
VIAGENS E ENTRETENIMENTO,SUSPENDED,2,13.33
VAREJO,SUSPENDED,21,13.29
SERVICOS RECORRENTES,SUSPENDED,3,12.00
ALIMENTACAO,SUSPENDED,35,11.18


### Estado

#### Quantidade de maus pagadores por estado

In [51]:
neutro_pagador_estado = gera_df_porcentagem(contratos_pagos_suspensos_merge, 'ESTADO')
neutro_pagador_estado

,contagem,porcentagem
ESTADO,,
SP,236,26.61
RJ,121,13.64
PR,62,6.99
MG,57,6.43
BA,51,5.75
SC,44,4.96
RS,44,4.96
PE,42,4.74
DF,39,4.40


#### Distribuição dos status de contrato por estado

In [52]:
neutro_pagador_estado_status = gera_df_porcentagem(contratos_pagos_suspensos_merge, 'ESTADO', 'STATUS_CONTRATO')
neutro_pagador_estado_status

contagem  porcentagem
tipo subtipo                         
TO   SETTLED           3        42.86
     ACTIVE            2        28.57
     SUSPENDED         2        28.57
SP   ACTIVE          116        49.15
     SETTLED          81        34.32
     SUSPENDED        39        16.53
SE   ACTIVE            5        41.67
     SETTLED           4        33.33
     SUSPENDED         3        25.00
SC   SETTLED          19        43.18
     ACTIVE           17        38.64
     SUSPENDED         8        18.18
RS   ACTIVE           23        52.27
     SETTLED          17        38.64
     SUSPENDED         4         9.09
RR   SETTLED           1       100.00
RO   SETTLED           2       100.00
RN   SETTLED           9        45.00
     ACTIVE            7        35.00
     SUSPENDED         4        20.00
RJ   ACTIVE           65        53.72
     SETTLED          43        35.54
     SUSPENDED        13        10.74
PR   ACTIVE           30        48.39
     SETTLED          22        35.48
     SUSPENDED        10        16.13
PI   ACTIVE            2        50.00
     SETTLED           2        50.00
PE   ACTIVE           23        54.76
     SETTLED          12        28.57
     SUSPENDED         7        16.67
PB   ACTIVE            8        61.54
     SETTLED           5        38.46
PA   SETTLED           8        53.33
     ACTIVE            4        26.67
     SUSPENDED         3        20.00
MT   SETTLED           6        54.55
     ACTIVE            4        36.36
     SUSPENDED         1         9.09
MS   ACTIVE            7        70.00
     SETTLED           2        20.00
     SUSPENDED         1        10.00
MG   ACTIVE           37        64.91
     SETTLED          14        24.56
     SUSPENDED         6        10.53
MA   ACTIVE            6        42.86
     SETTLED           5        35.71
     SUSPENDED         3        21.43
GO   SETTLED          11        42.31
     ACTIVE            9        34.62
     SUSPENDED         6        23.08
ES   ACTIVE            8        53.33
     SETTLED           6        40.00
     SUSPENDED         1         6.67
DF   ACTIVE           18        46.15
     SETTLED          18        46.15
     SUSPENDED         3         7.69
CE   ACTIVE           11        52.38
     SETTLED           7        33.33
     SUSPENDED         3        14.29
BA   ACTIVE           36        70.59
     SETTLED          12        23.53
     SUSPENDED         3         5.88
AP   ACTIVE            4        80.00
     SETTLED           1        20.00
AM   ACTIVE            4        66.67
     SETTLED           1        16.67
     SUSPENDED         1        16.67
AL   ACTIVE            3        60.00
     SUSPENDED         2        40.00
AC   SETTLED           3        75.00
     ACTIVE            1        25.00

Qual estado possui a maior taxa de contratos suspensos?

In [53]:
neutro_pagador_estado_status_concluido = extrair_subtipo_ordenado(neutro_pagador_estado_status, 'SUSPENDED')
neutro_pagador_estado_status_concluido

,,contagem,porcentagem
tipo,subtipo,,
AL,SUSPENDED,2,40.00
TO,SUSPENDED,2,28.57
SE,SUSPENDED,3,25.00
GO,SUSPENDED,6,23.08
MA,SUSPENDED,3,21.43
RN,SUSPENDED,4,20.00
PA,SUSPENDED,3,20.00
SC,SUSPENDED,8,18.18
AM,SUSPENDED,1,16.67


#### Distribuição de segmento de atuação por estado

In [54]:
neutro_pagador_estado_segmento = gera_df_porcentagem(contratos_pagos_suspensos_merge, 'ESTADO', 'SEGMENTO')
neutro_pagador_estado_segmento

contagem  porcentagem
tipo subtipo                                        
TO   VAREJO                           3        42.86
     ALIMENTACAO                      2        28.57
     BENS DURAVEIS                    1        14.29
     SERVICOS                         1        14.29
SP   ALIMENTACAO                    100        42.37
     SERVICOS                        47        19.92
     BENS DURAVEIS                   37        15.68
     VAREJO                          32        13.56
     SUPERMERCADO/FARMACIA            9         3.81
     SERVICOS RECORRENTES             7         2.97
     VIAGENS E ENTRETENIMENTO         4         1.69
SE   BENS DURAVEIS                    5        41.67
     ALIMENTACAO                      3        25.00
     SERVICOS                         2        16.67
     SUPERMERCADO/FARMACIA            1         8.33
     VAREJO                           1         8.33
SC   ALIMENTACAO                     20        45.45
     BENS DURAVEIS                    7        15.91
     VAREJO                           7        15.91
     SERVICOS                         5        11.36
     SUPERMERCADO/FARMACIA            4         9.09
     VIAGENS E ENTRETENIMENTO         1         2.27
RS   ALIMENTACAO                     16        36.36
     SERVICOS                         9        20.45
     VAREJO                           8        18.18
     BENS DURAVEIS                    4         9.09
     SERVICOS RECORRENTES             3         6.82
     SUPERMERCADO/FARMACIA            2         4.55
     VIAGENS E ENTRETENIMENTO         2         4.55
RR   BENS DURAVEIS                    1       100.00
RO   VAREJO                           2       100.00
RN   ALIMENTACAO                      6        30.00
     SERVICOS                         5        25.00
     VAREJO                           5        25.00
     BENS DURAVEIS                    2        10.00
     SUPERMERCADO/FARMACIA            2        10.00
RJ   ALIMENTACAO                     47        38.84
     VAREJO                          26        21.49
     SERVICOS                        17        14.05
     BENS DURAVEIS                   16        13.22
     SUPERMERCADO/FARMACIA            8         6.61
     SERVICOS RECORRENTES             3         2.48
     POSTO                            2         1.65
     VIAGENS E ENTRETENIMENTO         2         1.65
PR   ALIMENTACAO                     23        37.10
     SERVICOS                        20        32.26
     BENS DURAVEIS                    8        12.90
     VAREJO                           4         6.45
     POSTO                            3         4.84
     SUPERMERCADO/FARMACIA            3         4.84
     SERVICOS RECORRENTES             1         1.61
PI   BENS DURAVEIS                    2        50.00
     SUPERMERCADO/FARMACIA            1        25.00
     VAREJO                           1        25.00
PE   ALIMENTACAO                     15        35.71
     BENS DURAVEIS                    8        19.05
     VAREJO                           8        19.05
     SERVICOS                         4         9.52
     VIAGENS E ENTRETENIMENTO         4         9.52
     SUPERMERCADO/FARMACIA            2         4.76
     POSTO                            1         2.38
PB   SERVICOS                         4        30.77
     BENS DURAVEIS                    3        23.08
     ALIMENTACAO                      2        15.38
     SERVICOS RECORRENTES             2        15.38
     SUPERMERCADO/FARMACIA            1         7.69
     VAREJO                           1         7.69
PA   ALIMENTACAO                      4        26.67
     BENS DURAVEIS                    4        26.67
     VAREJO                           3        20.00
     SERVICOS                         2        13.33
     SUPERMERCADO/FARMACIA            2        13.33
MT   ALIMENTACAO                      3        27.27
     SERVICOS                         2        18.18
   

### Status contrato

In [55]:
neutro_pagador_status_contrato = gera_df_porcentagem(contratos_pagos_suspensos_merge, 'STATUS_CONTRATO')
neutro_pagador_status_contrato

,contagem,porcentagem
STATUS_CONTRATO,,
ACTIVE,450,50.73
SETTLED,314,35.40
SUSPENDED,123,13.87


## Análise Geral

### Tipo de empresa

In [60]:
geral_tipo_empresa = gera_df_porcentagem_por_perfil_pagador(todos_clientes, 'TIPO_EMPRESA')
geral_tipo_empresa

contagem  porcentagem
tipo subtipo                       
PJ   BOM          3950        78.78
     MAU           561        11.19
     NEUTRO        503        10.03
PF   BOM           580        68.88
     MAU           170        20.19
     NEUTRO         92        10.93
MEI  BOM          2136        76.39
     MAU           368        13.16
     NEUTRO        292        10.44

### Estado

In [61]:
geral_estado = gera_df_porcentagem_por_perfil_pagador(todos_clientes, 'ESTADO')
geral_estado.head(10)

contagem  porcentagem
tipo subtipo                       
TO   BOM            25        65.79
     NEUTRO          7        18.42
     MAU             6        15.79
SP   BOM          1838        78.08
     MAU           280        11.89
     NEUTRO        236        10.03
SE   BOM            64        79.01
     NEUTRO         12        14.81
     MAU             5         6.17
SC   BOM           395        80.94

Quais estados possuem as maiores taxa de bons pagadores?

In [62]:
geral_estado_bom = extrair_subtipo_ordenado(geral_estado, 'BOM')
geral_estado_bom.head()

,,contagem,porcentagem
tipo,subtipo,,
RO,BOM,31,81.58
MT,BOM,97,81.51
SC,BOM,395,80.94
PR,BOM,620,80.21
RS,BOM,360,80.00


Quais estados possuem as maiores taxes de maus pagadores?

In [63]:
geral_estado_mau = extrair_subtipo_ordenado(geral_estado, 'MAU')
geral_estado_mau.head()

,,contagem,porcentagem
tipo,subtipo,,
RR,MAU,5,20.83
AM,MAU,16,19.28
CE,MAU,44,17.25
DF,MAU,48,16.67
MA,MAU,14,16.28


### Segmento

In [64]:
geral_segmento = gera_df_porcentagem_por_perfil_pagador(todos_clientes, 'SEGMENTO')
geral_segmento

contagem  porcentagem
tipo                     subtipo                       
VIAGENS E ENTRETENIMENTO BOM            91        73.98
                         MAU            17        13.82
                         NEUTRO         15        12.20
VAREJO                   BOM          1180        75.06
                         MAU           234        14.89
                         NEUTRO        158        10.05
SUPERMERCADO/FARMACIA    BOM           627        82.39
                         MAU            75         9.86
                         NEUTRO         59         7.75
SERVICOS RECORRENTES     BOM           139        70.20
                         MAU            34        17.17
                         NEUTRO         25        12.63
SERVICOS                 BOM          1022        73.95
                         MAU           200        14.47
                         NEUTRO        160        11.58
POSTO                    BOM            76        79.17
                         MAU            11        11.46
                         NEUTRO          9         9.38
BENS DURAVEIS            BOM          1057        75.45
                         MAU           196        13.99
                         NEUTRO        148        10.56
ALIMENTACAO              BOM          2474        79.32
                         MAU           332        10.64
                         NEUTRO        313        10.04

Quais são os segmentos que costumam ter mais bons pagadores?

In [65]:
geral_segmento_bom = extrair_subtipo_ordenado(geral_segmento, 'BOM')
geral_segmento_bom.head()

,,contagem,porcentagem
tipo,subtipo,,
SUPERMERCADO/FARMACIA,BOM,627,82.39
ALIMENTACAO,BOM,2474,79.32
POSTO,BOM,76,79.17
BENS DURAVEIS,BOM,1057,75.45
VAREJO,BOM,1180,75.06


Quais são os segmentos que costumam ter mais maus pagadores?

In [66]:
geral_segmento_mau = extrair_subtipo_ordenado(geral_segmento, 'MAU')
geral_segmento_mau.head()

,,contagem,porcentagem
tipo,subtipo,,
SERVICOS RECORRENTES,MAU,34,17.17
VAREJO,MAU,234,14.89
SERVICOS,MAU,200,14.47
BENS DURAVEIS,MAU,196,13.99
VIAGENS E ENTRETENIMENTO,MAU,17,13.82


## Curva de acionamentos

In [67]:
def extrai_data_de_quitacao_contratos(dataframe_values):
    data_quitacao_contrato = {}
    contratos_unicos = set()
    for contrato_id, dt_ref_portfolio, status in dataframe_values:
        contratos_unicos.add(contrato_id)

        if status == "SETTLED":
            data_quitacao_contrato[contrato_id] = dt_ref_portfolio
    
    contratos_nao_quitados = contratos_unicos - data_quitacao_contrato.keys()
    for contrato in contratos_nao_quitados:
        data_quitacao_contrato[contrato] = None
    
    return pd.DataFrame(data_quitacao_contrato.items(), columns=['CONTRATO_ID','DT_QUITACAO']).set_index('CONTRATO_ID')

def extrai_sucessos_por_qnt_acionamentos(dataframe_values, contratos_indices_dict, data_quitacao_contrato):

    sucessos_por_qnt_acionamentos = defaultdict(set)

    for contrato, indices_contrato in contratos_indices_dict.items():
        _, _, dt_acao_portfolio, *_ = dataframe_values[indices_contrato].transpose()

        if data_quitacao_contrato[contrato] is not None:
            
            qnt_acionamentos = len(np.where(dt_acao_portfolio <= data_quitacao_contrato[contrato])[0])

            assert qnt_acionamentos % 2 == 0
            
            sucessos_por_qnt_acionamentos[qnt_acionamentos//2].add(contrato)
        else:
            sucessos_por_qnt_acionamentos['PENDENTE'].add(contrato)

    assert sucessos_por_qnt_acionamentos.get(0) is None

    contratos_quitados_sem_acionamento = data_quitacao_contrato.keys() - contratos_indices_dict.keys()
    sucessos_por_qnt_acionamentos[0] = contratos_quitados_sem_acionamento

    sucessos_por_qnt_acionamentos_ordenado = {key:len(sucessos_por_qnt_acionamentos[key]) for key in sorted(sucessos_por_qnt_acionamentos.keys() - {'PENDENTE'})}
    sucessos_por_qnt_acionamentos_ordenado['PENDENTE'] = len(sucessos_por_qnt_acionamentos['PENDENTE'])
    
    return sucessos_por_qnt_acionamentos_ordenado

def gera_soma_acumulada(dicionario):
    chaves_sem_pendente = dicionario.keys() - {'PENDENTE'}

    soma = 0
    dicionario_acumulado = {}
    for chave in sorted(chaves_sem_pendente):
        soma += dicionario[chave]
        dicionario_acumulado[chave] = soma
    
    dicionario_acumulado['PENDENTE'] = dicionario['PENDENTE']

    return dicionario_acumulado

def gera_mapeamento_contrato_indices(dataframe):

    contrato_indices_dict = defaultdict(list)
    
    for idx, (contrato_id, *_) in enumerate(dataframe):
        contrato_indices_dict[contrato_id].append(idx)
    
    return contrato_indices_dict

def gera_curva_acionamentos(dataframe_values, contratos_indices_dict, data_quitacao_contrato):

    start = time()
    sucessos_por_qnt_acionamento = extrai_sucessos_por_qnt_acionamentos(dataframe_values, contratos_indices_dict, data_quitacao_contrato)
    sucessos_por_qnt_acionamento_df = pd.DataFrame(sucessos_por_qnt_acionamento.items(), columns=['NUM_ACIONAMENTOS', 'QNT_QUITADOS']).head(-1)
    
    total_contratos = sucessos_por_qnt_acionamento_df.QNT_QUITADOS.sum()
    sucessos_por_qnt_acionamento_df['PORCENTAGEM_QUITADOS'] = sucessos_por_qnt_acionamento_df.QNT_QUITADOS / total_contratos * 100
    #print(f'Extrai {time()-start}')

    start = time()
    sucessos_acumulados_por_qnt_acionamentos = gera_soma_acumulada(sucessos_por_qnt_acionamento)
    sucessos_acumulados_por_qnt_acionamentos_df = pd.DataFrame(sucessos_acumulados_por_qnt_acionamentos.items(), columns=['NUM_ACIONAMENTOS', 'QNT_QUITADOS_ACUMULADA']).head(-1)
    sucessos_acumulados_por_qnt_acionamentos_df['PORCENTAGEM_QUITADOS_ACUMULADA'] = sucessos_acumulados_por_qnt_acionamentos_df.QNT_QUITADOS_ACUMULADA / total_contratos * 100
    #print(f'Soma {time()-start}')

    return sucessos_por_qnt_acionamento_df, sucessos_acumulados_por_qnt_acionamentos_df



In [68]:
#data_quitacao_contrato = extrai_data_de_quitacao_contratos(port_geral_contrato_status.values)
#data_quitacao_contrato.to_csv('data_quitacao_contrato.csv', index=True)

### Geral

In [69]:
port_comunicados_values = port_comunicados.values
port_comunicado_mapeamento = gera_mapeamento_contrato_indices(port_comunicados_values)

In [70]:
sucessos_por_qnt_acionamento_df , sucessos_acumulados_por_qnt_acionamentos_df = gera_curva_acionamentos(port_comunicados_values, port_comunicado_mapeamento, data_quitacao_contrato)

#### Quantidade / Porcentagem de contratos que foram quitados com exatamente X acionamentos

In [71]:
sucessos_por_qnt_acionamento_df.head()

,NUM_ACIONAMENTOS,QNT_QUITADOS,PORCENTAGEM_QUITADOS
0,0,2554,29.601298
1,1,934,10.825220
2,2,535,6.200742
3,3,462,5.354659
4,4,393,4.554937


#### Quantidade / Porcentagem acumulada de contratos quitados em até X acionamentos

In [72]:
sucessos_acumulados_por_qnt_acionamentos_df.head()

,NUM_ACIONAMENTOS,QNT_QUITADOS_ACUMULADA,PORCENTAGEM_QUITADOS_ACUMULADA
0,0,2554,29.601298
1,1,3488,40.426518
2,2,4023,46.627260
3,3,4485,51.981919
4,4,4878,56.536857


### Bom Pagador

In [77]:
ids_contratos_bons_pagadores = set(clientes_com_contratos_pago.CONTRATO_ID.unique())
port_comunicados_bom_pagador = port_comunicados[port_comunicados.CONTRATO_ID.isin(ids_contratos_bons_pagadores)]

In [81]:
len(port_comunicados_bom_pagador)

109798

In [83]:
port_comunicados_bom_pagador_values = port_comunicados_bom_pagador.values
port_comunicado_bom_pagador_mapeamento = gera_mapeamento_contrato_indices(port_comunicados_bom_pagador_values)

In [84]:
sucessos_por_qnt_acionamento_bom_pagador_df , sucessos_acumulados_por_qnt_acionamentos_bom_pagador_df = gera_curva_acionamentos(port_comunicados_bom_pagador_values, port_comunicado_bom_pagador_mapeamento, data_quitacao_contrato)

#### Quantidade / Porcentagem de contratos que foram quitados com exatamente X acionamentos

In [89]:
sucessos_por_qnt_acionamento_bom_pagador_df.head()

,NUM_ACIONAMENTOS,QNT_QUITADOS,PORCENTAGEM_QUITADOS
0,0,9646,66.045875
1,1,822,5.628210
2,2,466,3.190688
3,3,408,2.793564
4,4,346,2.369052


#### Quantidade / Porcentagem acumulada de contratos quitados em até X acionamentos

In [95]:
sucessos_acumulados_por_qnt_acionamentos_bom_pagador_df.head()

,NUM_ACIONAMENTOS,QNT_QUITADOS_ACUMULADA,PORCENTAGEM_QUITADOS_ACUMULADA
0,0,9646,66.045875
1,1,10468,71.674084
2,2,10934,74.864772
3,3,11342,77.658336
4,4,11688,80.027388


### Mau Pagador

Essa curva não faz sentido para os maus pagadores, pois eles não possuem contratos quitados.

## Exportação de dados - PRECISA ATUALIZAR

In [1]:
clientes_com_contratos_suspensos[['CONTRATO_ID','DT_REF_PORTFOLIO']].drop_duplicates().merge(port_comunicados, how='inner', on=['CONTRATO_ID','DT_REF_PORTFOLIO']).groupby('CONTRATO_ID').size().quantile(0.99)

NameError: name 'clientes_com_contratos_suspensos' is not defined

Prazo/parcelas do contrato da linha de crédito, geralmente o prazo médio é de 8 meses.

https://www.bndes.gov.br/wps/portal/site/home/financiamento/guia/prazos-periodicidade-pagamento

In [31]:
lst_nr_pago_sem_suspenso = list(lst_nr_pago_sem_suspenso)

In [34]:
mau_pagador_estado.to_csv('mau_pagador_estado.csv')
mau_pagador_status_contrato.to_csv('mau_pagador_status_contrato.csv')
mau_pagador_segmento.to_csv('mau_pagador_segmento.csv')
mau_pagador_tipo_empresa.to_csv('mau_pagador_tipo_empresa.csv')
bom_pagador_estado.to_csv('bom_pagador_estado.csv')
bom_pagador_status_contrato.to_csv('bom_pagador_status_contrato.csv')
bom_pagador_segmento.to_csv('bom_pagador_segmento.csv')
bom_pagador_tipo_empresa.to_csv('bom_pagador_tipo_empresa.csv')

Estado atual do empréstimo, podendo ser "Desembolso confirmado (DisbursementConfirmed), Aceito (Accepted), Desembolso solicitado (DisbursementRequested), Ativo (Active) ou Quitado (Settled)".